In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import ast
import csv

from helper_functions import *

output should be 3 files: 1) walls_dict.csv, 2) object_list_x.csv, and 3) drift_ranges_x.csv

maybe only lengths of 150?

obstacles from y=50 onwards.

just 2 drift sections for a length of 150.

sample y_start from 50 - 130

## Loop for generating layouts

In [14]:
layout_length = 301 #151

# walls stay the same
dict_keys = [str(x) for x in range(1, layout_length)]
walls_dict = dict.fromkeys(dict_keys, [0, 41])
with open('walls_dict.csv','w') as f:
    w = csv.writer(f)
    w.writerows(walls_dict.items())

# obstacles
num_entries = 5  # number of obstacles per drift section
# x ranges stay the same
x_values = list(range(2, 39))

# drift sections
safety_distance = 30
drift_range = 15

#for layout in range(1, 81):
for layout in range(1, 2):
    
    # generate drift sections
    ## 1st section
    y_start_1 = random.randint(50, layout_length-21)
    y_end_1 = y_start_1 + drift_range
    
    ## 2nd section
    y_start_2 = None

    while y_start_2 is None:
        y_start_2 = random.randint(50, layout_length-21)
        ranges = [(y_start_1-safety_distance, y_end_1+safety_distance)]
        if any(lower <= y_start_2 <= upper for (lower, upper) in ranges):
            y_start_2 = None
    y_end_2 = y_start_2 + drift_range
    
    ## 3rd section
    y_start_3 = None

    while y_start_3 is None:
        y_start_3 = random.randint(50, layout_length-21)
        ranges = [(y_start_1-safety_distance, y_end_1+safety_distance), (y_start_2-safety_distance, y_end_2+safety_distance)]
        if any(lower <= y_start_3 <= upper for (lower, upper) in ranges):
            y_start_3 = None
    y_end_3 = y_start_3 + drift_range
    
    ## 4th section
    y_start_4 = None

    while y_start_4 is None:
        y_start_4 = random.randint(50, layout_length-21)
        ranges = [(y_start_1-safety_distance, y_end_1+safety_distance), (y_start_2-safety_distance, y_end_2+safety_distance), (y_start_3-safety_distance, y_end_3+safety_distance)]
        if any(lower <= y_start_4 <= upper for (lower, upper) in ranges):
            y_start_4 = None
    y_end_4 = y_start_4 + drift_range

    drift_section_1 = [y_start_1, y_end_1, random.choice([-1, 1]), True, False]
    drift_section_2 = [y_start_2, y_end_2, random.choice([-1, 1]), True, False]
    drift_section_3 = [y_start_3, y_end_3, random.choice([-1, 1]), True, False]
    drift_section_4 = [y_start_4, y_end_4, random.choice([-1, 1]), True, False]
    
    drift_sections = [drift_section_1, drift_section_2, drift_section_3, drift_section_4]
    print(drift_sections)
    
    with open(f'drift_ranges_{layout}.csv','w') as f:
        w = csv.writer(f)
        w.writerows(drift_sections)
    
    
    # generate obstacles (dicts in list)
    # 5 for every drift section
    
    # define ranges for y
    y_values = list(range(y_start_1, y_end_1)) 
    obstacles_1 = [{'x': random.choice(x_values), 'y': random.choice(y_values), 'size': 2} for _ in range(num_entries)]

    y_values = list(range(y_start_2, y_end_2)) 
    obstacles_2 = [{'x': random.choice(x_values), 'y': random.choice(y_values), 'size': 2} for _ in range(num_entries)]

    y_values = list(range(y_start_3, y_end_3)) 
    obstacles_3 = [{'x': random.choice(x_values), 'y': random.choice(y_values), 'size': 2} for _ in range(num_entries)]

    y_values = list(range(y_start_4, y_end_4)) 
    obstacles_4 = [{'x': random.choice(x_values), 'y': random.choice(y_values), 'size': 2} for _ in range(num_entries)]

    # Combine dictionaries with unique keys
    combined_obstacles_dict = {}
    counter = 0

    for d_list in [obstacles_1, obstacles_2, obstacles_3, obstacles_4]:
        for item in d_list:
            combined_obstacles_dict[counter] = item
            counter += 1

    keys = combined_obstacles_dict[0].keys()

    with open(f'object_list_{layout}.csv','w') as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writerows(combined_obstacles_dict.values())

[[158, 173, 1, True, False], [67, 82, -1, True, False], [210, 225, 1, True, False], [264, 279, -1, True, False]]


## Helper functions

In [126]:
def get_walls(filename: str):
    """
    :param filename: filename of .txt file containing positions for all wall tiles. file must be in logs directory
    :return simple_dict: a dict easily accessible to grab individual positions of wall tiles
    """
    complete_path = os.getcwd() + '/logs/' + filename
    
    walls_df = pd.read_csv(complete_path, usecols=[1], header=None)
    walls_dict = walls_df.to_dict(orient='index')
    
    simple_dict = {key+1: ast.literal_eval(value[1]) for key, value in walls_dict.items()}  # +1 for starting at 1
    
    return simple_dict


def adjust_walls(walls_dict, scaling, edge):
    """
    :param walls_dict:
    :param scaling: int to scale up on-screen visualization
    :return updated walls_dict
    """
    for i in range(1, len(walls_dict)+1):
        walls_dict[i][0] = (walls_dict[i][0] + edge) * scaling
        walls_dict[i][1] = ((walls_dict[i][1] - 1) + edge) * scaling
        
    return walls_dict

In [31]:
def get_obstacles(filename: str, colnames=['x', 'y', 'size']):
    
    complete_path = os.getcwd() + '/logs/' + filename
    
    obstacles_df = pd.read_csv(complete_path, names=colnames, header=None)
    obstacles_dict = obstacles_df.to_dict(orient='index')
    
    return obstacles_dict


def adjust_obstacles(obstacles_dict, scaling, edge):
    """
    :param
    :return
    """
    for i in obstacles_dict:
        obstacles_dict[i]['x'] = ((obstacles_dict[i]['x'] - 1) + edge) * scaling
        obstacles_dict[i]['y'] = (obstacles_dict[i]['y'] - 1) * scaling
        obstacles_dict[i]['size'] = obstacles_dict[i]['size'] * scaling
    return obstacles_dict

In [122]:
def get_drift_sections(filename: str, colnames=['y_start', 'y_end', 'direction', 'visibility', 'fake']):
    
    complete_path = os.getcwd() + '/logs/' + filename
    drift_df = pd.read_csv(complete_path, names=colnames, header=None)

    drift_dict = drift_df.to_dict(orient='index')
    
    return drift_dict


def adjust_drift_sections(drift_dict, scaling):
    """
    param
    return
    """
    for i in drift_dict:
        drift_dict[i]['y_start'] = (drift_dict[i]['y_start'] - 1) * scaling
        drift_dict[i]['y_end'] = (drift_dict[i]['y_end'] - 1) * scaling
    return drift_dict